In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json
import os
import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import keras
from keras.layers import Conv2D, BatchNormalization, Activation, ZeroPadding2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, SeparableConv2D
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import *
from keras.models import load_model

In [0]:
def cutout(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [3]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [4]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation = None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        if self.augmentation is not None:
          image = self.augmentation.flow(image, shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [6]:
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state = 42)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}

In [0]:
any_img=cv2.imread(train_df.iloc[0][0])
input_shape = any_img.shape
inputs = Input(shape=input_shape)

x = Conv2D(32, 3, strides = 2,activation='relu', use_bias=False)(inputs)
x=BatchNormalization()(x)
x=Conv2D(3, 1,activation='relu', use_bias=False)(x)
x=BatchNormalization()(x)
x=Conv2D(32, 3, strides = 2,activation='relu', use_bias=False)(x)
x=BatchNormalization()(x)

x=SeparableConv2D(64, 3,activation='relu', use_bias=False, padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(0.15)(x)
x=SeparableConv2D(64, 3,activation='relu', use_bias=False, padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(0.15)(x)
x=SeparableConv2D(64, 3,activation='relu', use_bias=False, padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(0.15)(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=SeparableConv2D(128, 3,activation='relu', use_bias=False, padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(0.15)(x)
x=SeparableConv2D(128, 3,activation='relu', use_bias=False, padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(0.15)(x)
x=SeparableConv2D(128, 3,activation='relu', use_bias=False, padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(0.15)(x)
x=MaxPooling2D(pool_size=(2, 2))(x)

def build_tower(in_layer):
    x=SeparableConv2D(256, 3,activation='relu', use_bias=False, padding='same')(in_layer)
    x=BatchNormalization()(x)
    x=Dropout(0.15)(x)
    x=SeparableConv2D(256, 3,activation='relu', use_bias=False, padding='same')(x)
    x=BatchNormalization()(x)
    x=Dropout(0.15)(x)
    x=SeparableConv2D(512, 3,activation='relu', use_bias=False, padding='same')(x)
    x=BatchNormalization()(x)
    x=Dropout(0.15)(x)
    x=SeparableConv2D(1024, 3,activation='relu', use_bias=False)(x)
    x=BatchNormalization()(x)
    neck=Dropout(0.15)(x)
    neck=GlobalAveragePooling2D()(neck)
    neck = Dropout(0.2)(neck)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

gender = build_head("gender", build_tower(x))
image_quality = build_head("image_quality", build_tower(x))
age = build_head("age", build_tower(x))
weight = build_head("weight", build_tower(x))
bag = build_head("bag", build_tower(x))
footwear = build_head("footwear", build_tower(x))
emotion = build_head("emotion", build_tower(x))
pose = build_head("pose", build_tower(x))


model = Model(
    inputs=inputs, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
def validation_score(model):
    results = model.evaluate_generator(valid_gen, verbose=1)
    accuracies = {}
    losses = {}
    for k, v in zip(model.metrics_names, results):
        if k.endswith('acc'):
            accuracies[k] = round(v * 100, 4) 
        else:
            losses[k] = v
    return accuracies, losses

In [0]:
def process(model, epoches, lr, name, aug = None):
  train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation = aug) #ImageDataGenerator(horizontal_flip = True, vertical_flip = True)) #channel_shift_range=20, rotation_range= 10, preprocessing_function= cutout(v_l=0, v_h=1)))
  valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

  K.set_value(model.optimizer.lr, lr)

  save_dir = '/content/gdrive/My Drive/Models/my_models'
  model_name = name
  filepath = os.path.join(save_dir, model_name)
  checkpoint = ModelCheckpoint(filepath=filepath,
                              verbose=1,
                              save_best_only=True)
  callbacks=[checkpoint]

  model_info=model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=epoches,
    verbose=1,
    callbacks=callbacks)

In [0]:
aug = ImageDataGenerator(horizontal_flip = True, channel_shift_range=20, rotation_range= 10, preprocessing_function= cutout(v_l=0, v_h=1))

In [0]:
process(model, 20, 0.01, 'v1.h5', aug)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
360/360 [==============================] - 180s 499ms/step - loss: 8.0307 - gender_output_loss: 0.6954 - image_quality_output_loss: 1.0066 - age_output_loss: 1.4623 - weight_output_loss: 1.0218 - bag_output_loss: 0.9381 - footwear_output_loss: 1.0087 - pose_output_loss: 0.9526 - emotion_output_loss: 0.9453 - gender_output_acc: 0.5520 - image_quality_output_acc: 0.5390 - age_output_acc: 0.3782 - weight_output_acc: 0.6252 - bag_output_acc: 0.5472 - footwear_output_acc: 0.5077 - pose_output_acc: 0.6122 - emotion_output_acc: 0.7073 - val_loss: 8.4501 - val_gender_output_loss: 0.7461 - val_image_quality_output_loss: 1.0310 - val_age_output_loss: 1.4911 - val_weight_output_loss: 1.0194 - val_bag_output_loss: 0.9539 - val_footwear_output_loss: 1.2575 - val_pose_output_loss: 0.9709 - val_emotion_output_loss: 0.9801 - val_gender_output_acc: 0.5539 - val_image_quality_output_acc: 0.5625 - v

In [0]:
process(model, 20, 0.01, 'v2.h5', aug)

Epoch 1/20
360/360 [==============================] - 151s 419ms/step - loss: 6.5400 - gender_output_loss: 0.4083 - image_quality_output_loss: 0.8924 - age_output_loss: 1.3284 - weight_output_loss: 0.9364 - bag_output_loss: 0.8192 - footwear_output_loss: 0.7584 - pose_output_loss: 0.5381 - emotion_output_loss: 0.8589 - gender_output_acc: 0.8128 - image_quality_output_acc: 0.5721 - age_output_acc: 0.4153 - weight_output_acc: 0.6381 - bag_output_acc: 0.6350 - footwear_output_acc: 0.6665 - pose_output_acc: 0.7819 - emotion_output_acc: 0.7131 - val_loss: 6.5828 - val_gender_output_loss: 0.3865 - val_image_quality_output_loss: 0.9319 - val_age_output_loss: 1.3359 - val_weight_output_loss: 0.9370 - val_bag_output_loss: 0.7961 - val_footwear_output_loss: 0.7827 - val_pose_output_loss: 0.5118 - val_emotion_output_loss: 0.9010 - val_gender_output_acc: 0.8261 - val_image_quality_output_acc: 0.5595 - val_age_output_acc: 0.3962 - val_weight_output_acc: 0.6467 - val_bag_output_acc: 0.6557 - val_foo

In [0]:
model = load_model('/content/gdrive/My Drive/Models/my_models/v3.h5')
print(validation_score(model))

KeyboardInterrupt: ignored

In [0]:
aug = ImageDataGenerator(zoom_range = 0.2, horizontal_flip = True,rotation_range= 10, height_shift_range=0.4, preprocessing_function= cutout(v_l=0, v_h=1))

In [0]:
process(model, 50, 0.001, 'v4.h5', aug)

Epoch 1/50
360/360 [==============================] - 168s 468ms/step - loss: 7.0528 - gender_output_loss: 0.5159 - image_quality_output_loss: 0.8885 - age_output_loss: 1.3654 - weight_output_loss: 0.9474 - bag_output_loss: 0.8357 - footwear_output_loss: 0.9052 - pose_output_loss: 0.7222 - emotion_output_loss: 0.8725 - gender_output_acc: 0.7397 - image_quality_output_acc: 0.5769 - age_output_acc: 0.4139 - weight_output_acc: 0.6362 - bag_output_acc: 0.6183 - footwear_output_acc: 0.5777 - pose_output_acc: 0.6944 - emotion_output_acc: 0.7133 - val_loss: 6.5576 - val_gender_output_loss: 0.3466 - val_image_quality_output_loss: 0.8841 - val_age_output_loss: 1.3672 - val_weight_output_loss: 0.9330 - val_bag_output_loss: 0.8064 - val_footwear_output_loss: 0.8575 - val_pose_output_loss: 0.4576 - val_emotion_output_loss: 0.9052 - val_gender_output_acc: 0.8443 - val_image_quality_output_acc: 0.5837 - val_age_output_acc: 0.4012 - val_weight_output_acc: 0.6310 - val_bag_output_acc: 0.6447 - val_foo

Process ForkPoolWorker-736:
Process ForkPoolWorker-733:
Process ForkPoolWorker-734:
Process ForkPoolWorker-732:
Traceback (most recent call last):
Process ForkPoolWorker-731:
Process ForkPoolWorker-735:
Process ForkPoolWorker-726:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-729:
Process ForkPoolWorker-728:
Process ForkPoolWorker-727:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-725:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process

KeyboardInterrupt: ignored

In [0]:
model = load_model('/content/gdrive/My Drive/Models/my_models/v4.h5')
print(validation_score(model))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


31/31 [==============================] - 11s 351ms/step
({'gender_output_acc': 85.8367, 'image_quality_output_acc': 59.4254, 'age_output_acc': 40.5746, 'weight_output_acc': 63.5081, 'bag_output_acc': 65.2218, 'footwear_output_acc': 64.9194, 'pose_output_acc': 83.4173, 'emotion_output_acc': 69.6573}, {'loss': 6.433140370153612, 'gender_output_loss': 0.33663731188543383, 'image_quality_output_loss': 0.8803043923070354, 'age_output_loss': 1.3244499544943533, 'weight_output_loss': 0.930064162900371, 'bag_output_loss': 0.7998950269914442, 'footwear_output_loss': 0.8086108353830153, 'pose_output_loss': 0.4462944764283396, 'emotion_output_loss': 0.9068842068795235})


In [0]:
aug = ImageDataGenerator(zoom_range = 0.15, horizontal_flip = True, channel_shift_range=20, height_shift_range=0.3, rotation_range= 10)#, preprocessing_function= cutout(v_l=0, v_h=1))

In [0]:
process(model, 50, 0.001, 'v5.h5', aug)

Epoch 1/50
360/360 [==============================] - 158s 438ms/step - loss: 6.6892 - gender_output_loss: 0.4470 - image_quality_output_loss: 0.8713 - age_output_loss: 1.3253 - weight_output_loss: 0.9103 - bag_output_loss: 0.7978 - footwear_output_loss: 0.8519 - pose_output_loss: 0.6350 - emotion_output_loss: 0.8507 - gender_output_acc: 0.7852 - image_quality_output_acc: 0.5877 - age_output_acc: 0.4280 - weight_output_acc: 0.6439 - bag_output_acc: 0.6430 - footwear_output_acc: 0.6087 - pose_output_acc: 0.7328 - emotion_output_acc: 0.7137 - val_loss: 6.5174 - val_gender_output_loss: 0.3477 - val_image_quality_output_loss: 0.8925 - val_age_output_loss: 1.3562 - val_weight_output_loss: 0.9393 - val_bag_output_loss: 0.8079 - val_footwear_output_loss: 0.8396 - val_pose_output_loss: 0.4281 - val_emotion_output_loss: 0.9062 - val_gender_output_acc: 0.8574 - val_image_quality_output_acc: 0.5761 - val_age_output_acc: 0.3876 - val_weight_output_acc: 0.6346 - val_bag_output_acc: 0.6593 - val_foo

Process ForkPoolWorker-644:
Process ForkPoolWorker-645:
Process ForkPoolWorker-646:
Process ForkPoolWorker-647:
Process ForkPoolWorker-638:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-648:
Process ForkPoolWorker-649:
Traceback (most recent call last):
Process ForkPoolWorker-641:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multip


Epoch 00039: val_loss did not improve from 6.45614

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/iterator.py", line 116, in next
    return self._get_batches_of_transformed_samples(index_array)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 406, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py", line 153, in _get_batches_of_transformed_samples
    x.astype(self.dtype), params)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
KeyboardInterrupt
  File "/usr/lib/p

Epoch 40/50


  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 406, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 406, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/affine_transformations.py", line 333, in <listcomp>
    cval=cval) for x_channel in x]
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py", line 153, in _get_batches_of_transformed_samples
    x.astype(self.dtype), params)
  File "<ipython-input-6-9e56558388dc>", line 30, in __getitem__
    image = self.augmentation.flow(image, shuffle=False).next()
  File "/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py", line 464, in affine_transform
    if matrix.shape[0] != input

KeyboardInterrupt: ignored

In [54]:
model = load_model('/content/gdrive/My Drive/Models/my_models/v5.h5')
print(validation_score(model))

31/31 [==============================] - 8s 252ms/step
({'gender_output_acc': 86.1895, 'image_quality_output_acc': 57.6613, 'age_output_acc': 40.4738, 'weight_output_acc': 64.7681, 'bag_output_acc': 65.7258, 'footwear_output_acc': 63.9617, 'pose_output_acc': 84.1734, 'emotion_output_acc': 69.7581}, {'loss': 6.456139149204377, 'gender_output_loss': 0.3294422491904228, 'image_quality_output_loss': 0.8969590240909208, 'age_output_loss': 1.3380916310894875, 'weight_output_loss': 0.9271885541177565, 'bag_output_loss': 0.8078252653921804, 'footwear_output_loss': 0.8153151119908979, 'pose_output_loss': 0.4366447223771003, 'emotion_output_loss': 0.9046723977211983})


In [0]:
aug = ImageDataGenerator(zoom_range = 0.2,rotation_range= 10, horizontal_flip = True)

In [59]:
process(model, 10, 0.00001, 'v06.h5')

Epoch 1/10
360/360 [==============================] - 70s 196ms/step - loss: 5.4979 - gender_output_loss: 0.2368 - image_quality_output_loss: 0.8125 - age_output_loss: 1.1740 - weight_output_loss: 0.8209 - bag_output_loss: 0.6720 - footwear_output_loss: 0.6626 - pose_output_loss: 0.3417 - emotion_output_loss: 0.7775 - gender_output_acc: 0.9034 - image_quality_output_acc: 0.6186 - age_output_acc: 0.4898 - weight_output_acc: 0.6673 - bag_output_acc: 0.7181 - footwear_output_acc: 0.7169 - pose_output_acc: 0.8723 - emotion_output_acc: 0.7199 - val_loss: 6.3307 - val_gender_output_loss: 0.3102 - val_image_quality_output_loss: 0.9047 - val_age_output_loss: 1.3259 - val_weight_output_loss: 0.9186 - val_bag_output_loss: 0.7851 - val_footwear_output_loss: 0.7644 - val_pose_output_loss: 0.4168 - val_emotion_output_loss: 0.9047 - val_gender_output_acc: 0.8775 - val_image_quality_output_acc: 0.5620 - val_age_output_acc: 0.4098 - val_weight_output_acc: 0.6421 - val_bag_output_acc: 0.6749 - val_foot

In [0]:
# class CyclicLR(Callback):
#     def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
#                  gamma=1., scale_fn=None, scale_mode='cycle'):
#         super(CyclicLR, self).__init__()

#         self.base_lr = base_lr
#         self.max_lr = max_lr
#         self.step_size = step_size
#         self.mode = mode
#         self.gamma = gamma
#         if scale_fn == None:
#             if self.mode == 'triangular':
#                 self.scale_fn = lambda x: 1.
#                 self.scale_mode = 'cycle'
#             elif self.mode == 'triangular2':
#                 self.scale_fn = lambda x: 1/(2.**(x-1))
#                 self.scale_mode = 'cycle'
#             elif self.mode == 'exp_range':
#                 self.scale_fn = lambda x: gamma**(x)
#                 self.scale_mode = 'iterations'
#         else:
#             self.scale_fn = scale_fn
#             self.scale_mode = scale_mode
#         self.clr_iterations = 0.
#         self.trn_iterations = 0.
#         self.history = {}

#         self._reset()

#     def _reset(self, new_base_lr=None, new_max_lr=None,
#                new_step_size=None):
#         """Resets cycle iterations.
#         Optional boundary/step size adjustment.
#         """
#         if new_base_lr != None:
#             self.base_lr = new_base_lr
#         if new_max_lr != None:
#             self.max_lr = new_max_lr
#         if new_step_size != None:
#             self.step_size = new_step_size
#         self.clr_iterations = 0.
        
#     def clr(self):
#         cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
#         x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
#         if self.scale_mode == 'cycle':
#             return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
#         else:
#             return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
#     def on_train_begin(self, logs={}):
#         logs = logs or {}

#         if self.clr_iterations == 0:
#             K.set_value(self.model.optimizer.lr, self.base_lr)
#         else:
#             K.set_value(self.model.optimizer.lr, self.clr())        
            
#     def on_batch_end(self, epoch, logs=None):
        
#         logs = logs or {}
#         self.trn_iterations += 1
#         self.clr_iterations += 1

#         self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
#         self.history.setdefault('iterations', []).append(self.trn_iterations)

#         for k, v in logs.items():
#             self.history.setdefault(k, []).append(v)
        
#         K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
# clr = CyclicLR(base_lr=0.00001, max_lr=0.001,
#                         step_size=2520)
# model.fit_generator(generator=train_gen,validation_data=valid_gen,epochs=7,callbacks = [clr])

In [0]:
# h = clr.history
# lr = h['lr']
# acc_g = h['gender_output_acc']
# acc_i = h['image_quality_output_acc']
# acc_a = h['age_output_acc']
# acc_w = h['weight_output_acc']
# acc_b = h['bag_output_acc']
# acc_f = h['footwear_output_acc']
# acc_p = h['pose_output_acc']
# acc_e = h['emotion_output_acc']

In [0]:
# import seaborn as sns
# import matplotlib.pyplot as plt

In [0]:
# plt.figure(figsize=(20, 20))

In [0]:
model = load_model('/content/gdrive/My Drive/Models/my_models/v06.h5')

In [32]:
print(validation_score(model))

31/31 [==============================] - 5s 154ms/step
({'gender_output_acc': 87.752, 'image_quality_output_acc': 56.3508, 'age_output_acc': 41.0282, 'weight_output_acc': 64.1129, 'bag_output_acc': 67.3891, 'footwear_output_acc': 66.7339, 'pose_output_acc': 85.2319, 'emotion_output_acc': 69.8085}, {'loss': 6.326995818845687, 'gender_output_loss': 0.30972522977859746, 'image_quality_output_loss': 0.9019564140227533, 'age_output_loss': 1.3261381964529715, 'weight_output_loss': 0.9189572680381036, 'bag_output_loss': 0.7848065053263018, 'footwear_output_loss': 0.7638379383471704, 'pose_output_loss': 0.4167537804572813, 'emotion_output_loss': 0.9048204268178632})


In [0]:
# # losses = {
# # 	"gender_output": "binary_crossentropy",
# # 	"image_quality_output": "categorical_crossentropy",
# # 	"age_output": "categorical_crossentropy",
# # 	"weight_output": "categorical_crossentropy",

# }
loss_weights = {"gender_output": 0.5, "image_quality_output": 1.0, "age_output": 2.0, "weight_output": 1.0 , "bag_output": 1.0, "footwear_output": 1.0, "pose_output": 0.5 , "emotion_output": 2.0}
opt = SGD(lr=0.00001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [36]:
#this cell has been executed multiple times
process(model, 10, 0.00001, 'v07.h5',)

Epoch 1/10
360/360 [==============================]Epoch 1/10
 - 85s 237ms/step - loss: 7.1763 - gender_output_loss: 0.2294 - image_quality_output_loss: 0.8116 - age_output_loss: 1.1806 - weight_output_loss: 0.8222 - bag_output_loss: 0.6706 - footwear_output_loss: 0.6627 - pose_output_loss: 0.3470 - emotion_output_loss: 0.7798 - gender_output_acc: 0.9095 - image_quality_output_acc: 0.6181 - age_output_acc: 0.4896 - weight_output_acc: 0.6677 - bag_output_acc: 0.7151 - footwear_output_acc: 0.7135 - pose_output_acc: 0.8714 - emotion_output_acc: 0.7197 - val_loss: 8.1549 - val_gender_output_loss: 0.3018 - val_image_quality_output_loss: 0.8993 - val_age_output_loss: 1.3213 - val_weight_output_loss: 0.9141 - val_bag_output_loss: 0.7798 - val_footwear_output_loss: 0.7605 - val_pose_output_loss: 0.4148 - val_emotion_output_loss: 0.9001 - val_gender_output_acc: 0.8810 - val_image_quality_output_acc: 0.5605 - val_age_output_acc: 0.4108 - val_weight_output_acc: 0.6442 - val_bag_output_acc: 0.6754